### Langchain e llamacpp
Questo notebook contiene un esempio di utilizzo di langchain invocando un modello "locale" grazie a llamacpp.

E' necessario lanciare il server OPENAI utilizzando le istruzioni presenti nel repo llamacpp.

Grazie a llamacpp è possibile simulare "in locale" un server compatibile con le API OpenAI (vedi http://localhost:8000/docs)

In [1]:
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )


In [3]:
output = llm.invoke("Spiegami la meccanica quantistica in una sola frase")
print(output.content)

 La meccanica quantistica è l'approccio matematico alla fisica che descrive l'evoluzione di sistemi fisici microscopici tramite le relazioni probabilistiche, e non deterministiche, tra le azioni e le osservazioni.


Come nel notebook precedente, utilizzo la ChatCompletion API di OpenAI per interagire con il modello

In [5]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage)

messages =[
    SystemMessage(content="you are a fhisician and respond only in italian"),
    HumanMessage(content="Spiegami la meccanica quantistica")
]

output = llm.invoke(messages)
print(output.content)

 La meccanica quantistica è una branca della fisica che studia il comportamento dei sistemi fisici all'interno dell'universo, e le relazioni tra essi, a livello atomico e subatomico. È stata sviluppata come un'estensione dell'approccio classico alla fisica, e ha introdotto nuovi concetti e metodi per descrivere le proprietà delle particelle subatomiche, come l'elettrone e i quark.

La meccanica quantistica introduce la teoria delle funzioni di Schrödinger, che descrive i sistemi fisici attraverso l'evoluzione temporale di un'equazione di condizionale probabilità. Questa equazione descrive la probabilità di trovare un particolare stato di un sistema, e non la sua posizione esatta. La teoria di Heisenberg introduce l'impossibilità di misurare simultaneamente più proprietà di un sistema, e l'incertezza di misurazione.

La meccanica quantistica ha dato vita a molti importanti risultati scientifici, come l'equazione di Dirac per l'elettrone, la teoria del campo quantico e l'interpretazione 

### Streaming
Per gestire la risposta con lo streaminig

In [5]:
chunks = []
async for chunk in llm.astream("hello. tell me something about yourself"):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

 Hello! I'm an AI language model designed to assist users with a wide range of tasks, from answering questions to providing suggestions and recommendations. My main goal is to help you find the information you need quickly and easily. Is there anything specific you would like to know or ask?

### Caching

Memorizza le risposte alle invocazioni per poterle riutilizzare successivamente in modo da accorciare i tempi di risposta e ottimizzare la user experiece (e ridurre i costi di utilizzo dei modelli)

In langchain ci sono 2 modalità di cache:
- In memory cache
- SQLLite cache

In [6]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )

In [13]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

#prompt = 'Tell me a joke that a toddler can understand'
prompt = 'Raccontami una barzelletta che faccia ridere:'
llm.invoke(prompt)

CPU times: user 3.24 ms, sys: 3.18 ms, total: 6.42 ms
Wall time: 15.5 s


'\n\n"Mamma, mamma, ho perso il cucchiaio!"\n"Dove l\'avevi perso?"\n"Nell\'armadietto."\n"Che cosa stava facendo lì?"\n"Stava giocando a poker con i topolini."\n"Hai vinto?"\n"No, sono stato trickato dai topolini, ora devo andare a comprarne un nuovo."'

In [14]:
%%time
llm.invoke(prompt)

CPU times: user 854 µs, sys: 0 ns, total: 854 µs
Wall time: 867 µs


'\n\n"Mamma, mamma, ho perso il cucchiaio!"\n"Dove l\'avevi perso?"\n"Nell\'armadietto."\n"Che cosa stava facendo lì?"\n"Stava giocando a poker con i topolini."\n"Hai vinto?"\n"No, sono stato trickato dai topolini, ora devo andare a comprarne un nuovo."'

In [15]:
%%time
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


prompt = 'Tell me a joke that a toddler can understand'
llm.invoke(prompt)

CPU times: user 43.9 ms, sys: 21.6 ms, total: 65.5 ms
Wall time: 65.1 ms


'. Why did the chicken cross the road? To get to the other side! Why did the tomato turn red? Because it saw the salad dressing!'

In [16]:
%%time
llm.invoke(prompt)

CPU times: user 903 µs, sys: 0 ns, total: 903 µs
Wall time: 961 µs


'. Why did the chicken cross the road? To get to the other side! Why did the tomato turn red? Because it saw the salad dressing!'

### Streaming

In [17]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )

In [18]:
prompt = "Racconta una storia sulla luna e le stelle"
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

 Once upon a time, in a land far away, there was a young girl named Lily who loved nothing more than to gaze up at the night sky and marvel at the beautiful moon and stars that twinkled above her. She often lay on her back in the grass and lost herself in their beauty, dreaming of all the wondrous things that must exist beyond our world.

One night, as she was gazing at the moon, she noticed something strange. The moon seemed to be growing larger and larger until it filled almost the entire sky. She was so fascinated by this sight that she didn't even notice when a small spaceship landed nearby.

As she approached the spaceship, she saw a group of little creatures emerging from it. They were tiny and had big, round eyes and fluffy fur. They were the Moon People, and they had come to earth to help humans like Lily appreciate the beauty of the moon and stars.

The Moon People were amazed by the wonders of Earth, and they wanted to share their knowledge with humans. They taught Lily about

### PromptTemplates

Consentono di creare prompt dinamici da passare all'LLM. Permette di combinare testo statico con variabili dinamiche.

IN langchain ci sono:
- PromptTemplates (si possono utilizzare con qualsiasi LLM)
- ChatPromptTemplates (da utilizzare per conversazioni con LLM)

In [19]:
from langchain.prompts import PromptTemplate 

template = '''You are an expert vitologist. Write a few sentences about that virus {virus} in {language}'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='italian')

In [20]:
prompt

'You are an expert vitologist. Write a few sentences about that virus hiv in italian'

In [21]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        temperature=0
    )

output = llm.invoke(prompt)
print(output.content)

 Il virus HIV, o Human Immunodeficiency Virus, è un virus che attacca e distrugge le cellule dell'immunosistema umano, causando la malattia dell'AIDS (Sindrome dell'Immunodeficienza Scompensata). È trasmesso principalmente attraverso il contatto sessuale non protetto con il sperma infettato, ma anche attraverso l'inoculazione di sangue infettato, l'inoculazione di farmaci infettati e attraverso la trasmissione verticale da madre a figlio durante il parto.


### ChatPromptTemplate

Si può utilizzare con OpenAI Chatcompletion API

In [22]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content = 'You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} cities in  {country} by population order by population descending.')
    ]
)

messages = chat_template.format_messages(n = '10', country = 'Italy')
print(messages)

[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 10 cities in  Italy by population order by population descending.')]


In [23]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        temperature=0
    
    )
output = llm.invoke(messages)
print(output.content)

 {
"cities": [
{
"name": "Milan",
"population": 2693959
},
{
"name": "Rome",
"population": 2833020
},
{
"name": "Naples",
"population": 2187475
},
{
"name": "Florence",
"population": 1792946
},
{
"name": "Torino",
"population": 2335695
},
{
"name": "Bologna",
"population": 1087936
},
{
"name": "Genoa",
"population": 874443
},
{
"name": "Venice",
"population": 268983
},
{
"name": "Pisa",
"population": 187234
},
{
"name": "Reggio Emilia",
"population": 159682
}
]
}
